# Case Study: Modeling Bikeshare Usage

In this case study, we'll use regression techniques to study some of the factors that drive the usage of bikesharing systems. The data set for this case study was collected from the Capital Bikeshare system in Washington DC. We use the aggregated version graciously provided by the authors of the following paper: 

> Fanaee-T, Hadi, and Gama, Joao, "Event labeling combining ensemble detectors and background knowledge", Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg, doi:10.1007/s13748-013-0040-3.

This data set includes information about the season and time of year; the weather; and the count of bicycle users on each day for two years. This level of information gives us considerable ability to model phenomena in the data. 

In this case study, our primary aim will be **explanatory** modeling rather than **predictive** modeling. That is, we are more interested in learning something about the data we have than we are about predicting new, unseen data. To this end, we'll be prioritizing using a simple model with interpretable parameters. 

Some people would consider the emphasis on explanation vs. prediction to be the dividing line between *statistics* and *machine learning*, but in practice this is very fuzzy indeed. 

__Before we get started:__ Make sure the file bikeshare.csv is in the same folder as this notebook

In [1]:
#standard imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#read in the data
bikeshare = pd.read_csv("bikeshare.csv")


Okay, let's take a look

There are actually three columns we could aim to predict. *Casual* users don't sign up for the bikeshare service -- they just pay each time they decide to take a ride. In contrast, *registered* users pay a regular subscription fee. The `cnt` column is simply the sum of the casual and registered rides for each day. __For this case study, we will focus on modeling the behavior of casual riders.__ 

## Exploratory Analysis

Let's start by simply visualizing the total counts over time. For this, it helps to convert the `dteday` column into a proper `datetime` column.

In [6]:
import datetime
bikeshare['dteday']=pd.to_datetime(bikeshare['dteday'])

Now, let's make a simple plot

A few observations: 

- There seems to have been a slight overall increase in casual ridership over time. 
- There is a lot of variation in the data, with some days having very few riders and other days very many.  
- There's some seasonal variation -- for example, we see noticeable dips in winter. 

I wonder whether there's much of a difference between weekend vs. weekday ridership. 

More people use the bikeshare on the weekend! What if we breakdown by month?

This is kind of hard to read, let's make a plot!

Ok, we're beginning to see some patterns now. There's a clear increase in casual ridership on the weekends, and in warmer months in the middle of the year. So, we'd expect these variables to play major roles in our modeling.

We can break things up further -- what's the dependence on the weather? 

In [4]:
m=bikeshare.groupby(['workingday','weathersit','mnth'])[['casual']].mean()


Ok, this makes sense: cloudy weather doesn't seem to be much of a deterrent, but precipitation makes a major difference. 

## Data Prep

We've now learned quite a lot about what we're looking for, so let's go ahead and begin preparing our data for modeling. Let's first grab only the variables that we want to include as a new data frame. 

Note that we've excluded, for example, `season`, because all the information in the `season` column is actually already contained in the `mnth`. 

In [20]:
cols = ["casual", 
        "mnth", 
        "weathersit", 
        "workingday",
        "yr",
        "temp", 
        "hum", 
        "windspeed",
        "holiday"]

model_df=bikeshare[cols]

Next, we need to encode the `mnth` column in a way that an algorithm can understand. The easiest way is to use `pd.get_dummies()`, which will create a `0-1` column associated to each possible value. 

__Note:__ Using dummy variables like this is often called a one-hot encoding in the machine learning world.

Now we can separate our data into predictor and target variables. 

Model time! Because our interest is explanatory rather than predictive, we are just going to train and evaluate the model on the same set of data. There are still potential problems with doing this! In this particular case, however, we're not too worried about overfitting because the number of data points far outnumbers the number of model parameters.

LinearRegression()

Not bad! The model does a fair job of replicating the true data. The fit isn't perfect, in at least two ways. 

1. There are some visible discrepancies between the modeled and observed values. 
2. There are a few days on which the model predicts negative ridership, which obviously can't be correct. 

So, this model has room for improvement. Still, considering how simple it was to put together, it's a fair start! Recall that one of our main aims was to learn something about which conditions make a difference for ridership. Linear models are great for this, because we can just investigate the coefficients directly. 

Here's what we observe: 

1. Recall that higher values of the `weathersit` variable correspond to more severe weather. The negative coefficient indicates that, for each step up the severity scale, roughly 100 riders are lost. 
2. After controlling for the other variables, there are roughly 800 more riders on weekends than working days. Wow!
3. The `yr` coefficient accounts for the slight overall increase in riders over time: a day in year 1 might have nearly 300 more riders than a comparable day in year 0. 
4. Warm temperatures bring out the bicyclists! 
5. Similarly, no one likes humidity, or high winds. 
6. There tends to be a large drop in casual bicyclists on holidays. 

The various month results deserve special attention. Higher values indicate that, all else being equal, the model predicts more riders that month. However, we should be very cautious in interpreting these results, since certain months are likely to have lower temperatures or higher humidities, for example. So, it's not usually the case that "all else is equal." Still, these results do give us suggestive and potentially useful quantitative insights, such as rider's preferences for warmer months over colder ones, especially in the spring as the snow melts. 

## Next? 

Depending on our goals, we have many possible next steps. We might aim to improve our model accuracy by using more sophisticated models or collecting more data. We might aim to deploy more complex models in order to handle dependencies between the month variables and the weather variables, for example. Rather than the explanatory angle we've used here, we might instead choose to look at this problem through a predictive lens, and aim to predict the bikeshare usage in unseen, coming months. As usual in data science, each model opens at least as many doors as it closes. 

<figure class="image" style="width:100%">
  <img src="https://d33wubrfki0l68.cloudfront.net/795c039ba2520455d833b4034befc8cf360a70ba/558a5/diagrams/data-science-explore.png" alt="A diagram with the words import, tidy, transform, visualize, model, and communicate. Import points to tidy. Tidy points to transform. Transform points to visualize, visualize poitns to model, and model points to visualize, thus forming a directed cycle of arrows. This cycle collectively points to the final word, communicate." width="600px">
    <br>
    <caption><i>The cycle of data science. Image credit: Hadley Wickham</i></caption>
</figure>
